In [1]:
!pip install git+https://github.com/superannotateai/generated_text_detector.git@v1.1.0
from generated_text_detector.utils.text_detector import GeneratedTextDetector


detector = GeneratedTextDetector(
    "SuperAnnotate/ai-detector",
    device="cuda",
    preprocessing=True
)

text_example = "It's not uncommon for people to develop allergies or intolerances to certain foods as they get older. It's possible that you have always had a sensitivity to lactose (the sugar found in milk and other dairy products), but it only recently became a problem for you. This can happen because our bodies can change over time and become more or less able to tolerate certain things. It's also possible that you have developed an allergy or intolerance to something else that is causing your symptoms, such as a food additive or preservative. In any case, it's important to talk to a doctor if you are experiencing new allergy or intolerance symptoms, so they can help determine the cause and recommend treatment."

res = detector.detect_report(text_example)

print(res["generated_score"])


  Cloning https://github.com/superannotateai/generated_text_detector.git (to revision v1.1.0) to /tmp/pip-req-build-bq3lqz5q
  Running command git clone --filter=blob:none --quiet https://github.com/superannotateai/generated_text_detector.git /tmp/pip-req-build-bq3lqz5q
  Resolved https://github.com/superannotateai/generated_text_detector.git to commit dbd6317968d144291192a2baf33e11df4e6cdf65
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

0.732421875


In [2]:
import pandas as pd
import re
from html import unescape
from tqdm import tqdm
testing_frame = pd.read_parquet("/kaggle/input/dactyl-adversarial/finetuned-texts.parquet")
testing_frame = testing_frame[testing_frame.split == "testing"]

In [3]:
y_pred = list()
for text in tqdm(testing_frame["text"].values):
    res = detector.detect_report(text)
    y_pred.append(res["generated_score"])

100%|██████████| 5516/5516 [02:41<00:00, 34.20it/s]


In [4]:
'''
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm


pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

def data():
    for text in testing_frame["text"].values:
        yield text
y_pred = list()
for result in tqdm(pipe(data(),batch_size=16, max_length=512,truncation=True, return_all_scores=True), total=len(testing_frame)):
    print(result)
    for output in result:
        if output["label"] == "AI":
            y_pred.append(output["score"])
'''


'\n# Use a pipeline as a high-level helper\nfrom transformers import pipeline, AutoTokenizer\nfrom tqdm import tqdm\n\n\npipe = pipeline("text-classification", model=model, tokenizer=tokenizer)\n\ndef data():\n    for text in testing_frame["text"].values:\n        yield text\ny_pred = list()\nfor result in tqdm(pipe(data(),batch_size=16, max_length=512,truncation=True, return_all_scores=True), total=len(testing_frame)):\n    print(result)\n    for output in result:\n        if output["label"] == "AI":\n            y_pred.append(output["score"])\n'

In [5]:
testing_frame["superannotate_pred"] = y_pred
testing_frame = testing_frame.drop(columns=["text"])
testing_frame[["id","superannotate_pred","domain","model","target"]].to_csv(f"superannotate-results.csv",index=False)
